In [ ]:
# Ignorar sklearn warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='skimage')

# Librería de ámbito matemático, científico y de ingeniería. En nuestro caso, solo usamos la convolución.
import scipy.io

# Interfaz para HDF5. Solo necesario para cargar las anotaciones del dataset.
import hdf5storage

# Librería para la computación científica. En nuestro caso, la usamos por la representación de datos y hacer calculas de forma
# rápida.
import numpy as np

# Librerías de visión por computador
# OpenCV: Generalista.
# Skimage: Generalista.
# Dlib: Menos funcionalidades, muy rápido e implementaciones muy interesantes.
import dlib
import cv2
from skimage import io
from skimage import color
from skimage import filters
from skimage import util
from skimage import transform
from skimage import feature

# Librerías para gráficos en 2d/3d. En nuestro caso para mostrar las imágenes y gráficos en general.
from matplotlib import pyplot as plt
from IPython.display import HTML
from IPython.display import Video

# Necesario para visualizar los resultados en la misma página.
%matplotlib inline

# Introducción

Para comprender mejor las explicaciones de la presentación procederemos a realizar las siguientes tareas:

* __Detector de landmarks__: Entrenaremos desde cero un detector de puntos de referencia en los rostros, y lo pondremos a prueba con una serie de imágenes y videos.

* __Reconocimiento facial__: Usaremos un modelo entrenado para generar vectores que describan los rostros. Y con estos descriptores entrenaremos un clasificador que nos permitirá reconocer diferentes rostros, para finalizar probaremos el detector en un video.



# Entrenando un detector de landmarks

## Explorando el dataset

Antes de empezar vamos a recordar de qué forma se estructuraba nuestro dataset para obtener las anotaciones necesarias para entrenar nuestro detector.

Ruta del dataset:
__resources\face_dataset__.

In [ ]:
# Cargar las anotaciones del dataset.
anotaciones_path = "resources/session2/face_dataset/anno.mat"
anotaciones = hdf5storage.loadmat(anotaciones_path)
anotaciones = anotaciones["anno"]


In [ ]:
# Estructura de las anotaciones. Esta varia mucha según que de donde proviene el dataset.
print("Datos de una imagen:")
print("-"*20)
print("Nombre del fichero: ", anotaciones[0][0][0])
print("Bboxes:\n %s" % anotaciones[0][1][0][0])
print("Rotación de la cara en el eje [X, Y, Z]:\n %s" % anotaciones[0][2][0][0])
print("Landmarks:\n %s" % anotaciones[0][3][0][0])

In [ ]:
# Numero de muestras
print("Numero de imagenes:", len(anotaciones))
print("Numero de caras anotadas:", np.sum([len(anotacion[1][0]) for anotacion in anotaciones]))

In [ ]:
import random # Generar números aleatorios

# Seleccionar una imagen de forma aleatoria
random_image = random.randint(0, (len(anotaciones)))
filename = anotaciones[random_image][0][0][0]
bboxs = anotaciones[random_image][1][0]
landing_marks = anotaciones[random_image][3][0]

# Estructura
f, (ax0, ax1) = plt.subplots(1, 2, figsize=(20, 10))

# Abrir la imágen
img = io.imread("resources/session3/face_dataset/" + filename)
ax0.imshow(img);
ax0.set_title("Imagen sin anotar")

# Dibujar la información disponible
for bbox, bbox_landing_marks in zip(bboxs, landing_marks):
    # Rectangle
    (x1, y1), (x2, y2) = bbox.astype(int)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
    # Landmarks
    for landmark in bbox_landing_marks:
        x, y = landmark.astype(int)
        cv2.circle(img, (x, y) , 7, (255, 0, 0), -1)
        
# Mostrar la imagen
ax1.imshow(img);
ax1.set_title("Imagen anotada");

## Generando los xml

Tal y como hicimos en la última sesión necesitamos generar un fichero XML con un formato específico de para poder entrenar nuestro detector de landmarks.

In [ ]:
import xml.etree.cElementTree as ET # Necesario para generar el xml
import xml.dom.minidom as minidom # Necesario para que poder formatear el xml

def prettify(elem):
    """
    Devuelve el xml con un formato de string estructurado.
    
    Params
    ------
    elem : Et.Element
        Árbol xml.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")


In [ ]:
num_training_samples = 165

# Training set
root = ET.Element("dataset")
doc = ET.SubElement(root, "name").text = "Dataset deteccion de landmarks Curso Altran - Training"
images = ET.SubElement(root, "images")

for imagen in anotaciones[:165]:
    nombre_de_archivo = imagen[0][0][0]
    bboxes = imagen[1][0]
    landing_marks = imagen[3][0]
    
    imagen_actual = ET.SubElement(images, "image", file=nombre_de_archivo)
    
    for bbox, bbox_landing_marks in zip(bboxes, landing_marks):
        
        # Rectangle
        (x1, y1), (x2, y2) = bbox.astype(int)
        box = ET.SubElement(imagen_actual, "box", top=str(y1), left=str(x1), width=str(x2 - x1), height=str(y2 - y1))
        # Landmarks
        for idx in range(len(bbox_landing_marks)):
            x, y = bbox_landing_marks[idx].astype(int)
            ET.SubElement(box, "part", name=str(idx), x=str(x), y=str(y))
    
xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="   ")
with open("resources/session3/face_dataset/training_dataset.xml", "w") as f:
    f.write(xmlstr)

In [ ]:
# Test set
root = ET.Element("dataset")
doc = ET.SubElement(root, "name").text = "Dataset deteccion de landmarks Curso Altran - Training"
images = ET.SubElement(root, "images")

for imagen in anotaciones[:165]:
    nombre_de_archivo = imagen[0][0][0]
    bboxes = imagen[1][0]
    landing_marks = imagen[3][0]
    
    imagen_actual = ET.SubElement(images, "image", file=nombre_de_archivo)
    
    for bbox, bbox_landing_marks in zip(bboxes, landing_marks):
        
        # Rectangle
        (x1, y1), (x2, y2) = bbox.astype(int)
        box = ET.SubElement(imagen_actual, "box", top=str(y1), left=str(x1), width=str(x2 - x1), height=str(y2 - y1))
        # Landmarks
        for idx in range(len(bbox_landing_marks)):
            x, y = bbox_landing_marks[idx].astype(int)
            ET.SubElement(box, "part", name=str(idx), x=str(x), y=str(y))
    
xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="   ")
with open("resources/session3/face_dataset/test_dataset.xml", "w") as f:
    f.write(xmlstr)

# Entrenar el dataset automaticamente

Ahora que ya disponemos del dataset anotado en el formato que espera la librería Dlib, procedemos a entrenar nuestro detector con la implementación del proceso descrito en  __One Millisecond Face Alignment with an Ensemble of Regression Trees by
Vahid Kazemi and Josephine Sullivan, CVPR 2014__.


In [ ]:
options = dlib.shape_predictor_training_options()

# Este parámetro es muy útil si tenemos un dataset pequeño y queremos aumentar nuestros datos de entrenamiento.
options.oversampling_amount = 1

# Algunas opciones como el tree_depth o el nu se han escojido de tal forma que se reduce
# la capacidad del detector, pero el tiempo de entrenamiento es mucho menor.
options.num_threads = 3
options.nu = 0.05
options.tree_depth = 2
options.be_verbose = True

# Esta es la línea que ejecuta el entrenamiento y guarda los parámetros del modelo en el fichero especificado.
training_xml_path = "resources/session3/face_dataset/training_dataset.xml"
dlib.train_shape_predictor(training_xml_path, "resources/session3/predictor.dat", options)




In [ ]:
# Medimos la accuracy con el dataset de entrenamiento
print("\nAccuracy con el dataset de entrenamiento: {}".format(
    dlib.test_shape_predictor(training_xml_path, "resources/session3/landmark_detector.dat")))

# Medimos la accuracy con el dataset de test
testing_xml_path = os.path.join(faces_folder, "testing_with_face_landmarks.xml")
print("Accuracy con el dataset de test: {}".format(
    dlib.test_shape_predictor(testing_xml_path, "predictor.dat")))

## Resultados en una imagen
### Resultados con nuestro detector

In [ ]:
# Cargamos nuestra imagen de test favorita
final= io.imread("resources/session2/our_face_dataset/test/test.png")

# Cargamos el detector de caras basado en CNN y el detector de landmarks
facerec = dlib.cnn_face_detection_model_v1("resources/session2/mmod_human_face_detector.dat")
predictor = dlib.shape_predictor("resources/session3/predictor.dat")

#Inferimos los bbox con caras a partir del detector
bboxes = detector(final, 1)
print("Número de caras encontradas {}".format(len(bboxes))) 

# Por cada cara encontrada en la imagen ejecutamos el detector de landmarks y mostramos el resultado
for i, bbox in enumerate(bboxes):
    cv2.rectangle(final,(bbox.left(),bbox.top()),(bbox.right(),bbox.bottom()),(0,255,0),3)
    shape = predictor(img, d)
    print(shape)
    
plt.figure(figsize=(50, 50))
plt.imshow(final)

### Resultados con el detector de dlib

In [ ]:
# Cargamos nuestra imagen de test favorita
final= io.imread("resources/session2/our_face_dataset/test/test.png")

# Cargamos el detector de caras basado en CNN y el detector de landmarks
facedetect = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("resources/session3/shape_predictor_68_face_landmarks.dat")

#Inferimos los bbox con caras a partir del detector
bboxes = facedetect(final, 1)
print("Número de caras encontradas {}".format(len(bboxes))) 

# Por cada cara encontrada en la imagen ejecutamos el detector de landmarks y mostramos el resultado
for i, bbox in enumerate(bboxes):
    cv2.rectangle(final,(bbox.left(),bbox.top()),(bbox.right(),bbox.bottom()),(0,255,0),3)
    shape = predictor(final, bbox)
    for num_part in range(shape.num_parts):
        point = shape.part(num_part)
        cv2.circle(final, (point.x, point.y) , 2, (255, 0, 0), -1)
    
plt.figure(figsize=(50, 50))
plt.imshow(final)

## Resultados en un video


In [ ]:
# Definimos el objeto que nos permitira la lectura del vídeo
input_video = cv2.VideoCapture('resources/session3/test.mp4')

# Definimos el objeto que nos permitira la escritura del vídeo
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output_video = cv2.VideoWriter('resources/session3/output.mp4',fourcc, 25.0, (640,480))

# Cargamos el detector de caras basado en CNN y el detector de landmarks.
facedetect = dlib.get_frontal_face_detector()
facelandmarks = dlib.shape_predictor("resources/session3/shape_predictor_68_face_landmarks.dat")

# No sé por que OpenCV no detecta bien el final del video así que tenemos que usar estas variables extras.
ret = True
num_frame = 0

# Mientras hay frames disponibles en el video de entrada
while(input_video.isOpened() and ret):
    ret, frame = input_video.read()
    
    if ret:
        # Convertimos la imágen de BGR(OpenCV) a RGB(Dlib)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detectamos los rostros y dibujamos un bbox y círculos en los puntos de referéncia.
        bboxes = facedetect(image_rgb, 1)
        for i, bbox in enumerate(bboxes):
            # Bbox
            cv2.rectangle(image_rgb,(bbox.left(),bbox.top()),(bbox.right(),bbox.bottom()),(0,255,0),3)
            # Landmarks
            shape = facelandmarks(final, bbox)
            for num_part in range(shape.num_parts):
                point = shape.part(num_part)
                cv2.circle(final, (point.x, point.y) , 1, (255, 0, 0), -1)
        # Convertimos la imágen de RGB(Dlib) a BGR(OpenCV)        
        image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        # Guardamos en el disco
        output_video.write(image_rgb)
        
    # Mostrar frame
    num_frame += 1
    if num_frame % 100 == 0:
        print("Actual frame: %s" % (num_frame))
        print("Num caras en el frame: %s" % (len(bboxes)))
        
input_video.release()
output_video.release()

# Reconocimiento facial
Para realizar el reconocimiento facial tenemos que realizar básicamente los siguientes pasos:

* Detectar caras
* Normalizarlas/Alinearlas
* Embbedirlas
* Clasificarlas

## Normalización / Alineación de los rostros

Con este proceso intentamos normalizar las caras usando los landmarks, para ello aplicamos una transformación geométrica, que veremos en detalle en la siguiente sesión, y que nos permite centrar todos los rostros antes de calcular el vector embbedido.

In [ ]:
import sys
sys.path.insert(0, 'resources/session3/')
from align import AlignDlib

# Inicializa la clase especificando que detector de landmarks usaremos
alignment = AlignDlib('resources/session3/shape_predictor_68_face_landmarks.dat')
facedetect = dlib.get_frontal_face_detector()

# Abrimos la imagen
image = io.imread("resources/session3/face_dataset/2404040793.jpg")

#Detectamos los rostros
bboxes = facedetect(image, 1)

# Alineación
(image_aligned, landmarks) = alignment.align(80, image, bboxes[0], landmarkIndices=AlignDlib.INNER_EYES_AND_BOTTOM_LIP)

# Imagen original
plt.subplot(131)
plt.imshow(image)

# Recorte
plt.subplot(132)
bbox = bboxes[0]
plt.imshow(image[bbox.top():bbox.bottom(), bbox.left():bbox.right()])

# Imagen centrada
plt.subplot(133)
plt.imshow(image_aligned)

# Calculo del vector embedded

Ahora toca convertir nuestras imágenes a un descriptor que define de la forma más certera los rostros. Este proceso es arduo de entrenar, ya que para conseguir unos resultados equivalentes a los que veremos ahora necesitaríamos una GPU de última generación, 3 millones de rostros anotados y más de 24 h de cómputo. Es por eso que vamos a usar una red preentrenada y simplemente vamos a usar los descriptores para clasificar los rostros que nosotros queramos.

In [ ]:
facedetector = dlib.get_frontal_face_detector()
facealignment = dlib.shape_predictor('resources/session3/shape_predictor_68_face_landmarks.dat')
facerembedder = dlib.face_recognition_model_v1("resources/session3/dlib_face_recognition_resnet_model_v1.dat")

# Abrimos una imagen cualquiera
image = io.imread("resources/session3/face_dataset/4553922208.jpg")

# Detectamos las caras
bboxes = facedetector(image)

# Iteramos sobre las caras encontradas y calculamos el vector
for k, bbox in enumerate(bboxes):
    landmarks = facealignment(image, bbox)
    face_descriptor = facerembedder.compute_face_descriptor(image_aligned, landmarks)
    print("Rostro %s : %s" % (k, np.array(face_descriptor)))


# Caso aplicado

En el video usado anteriormente buscaremos rostros en los primeros n frames, generaremos sus vectores y usaremos el famoso algoritmo PCA para reducir los vectores de 128 dimensiones a solo dos. Para finalizar mostraremos los descriptores en un gráfico y usaremos el algoritmo DBScan para encontrar los diferentes clúster y aproximar el número de caras diferentes que aparecen en el video.

In [ ]:
# Detectores
facedetector = dlib.get_frontal_face_detector()
facelandmark = dlib.shape_predictor('resources/session3/shape_predictor_68_face_landmarks.dat')
facerembedder = dlib.face_recognition_model_v1("resources/session3/dlib_face_recognition_resnet_model_v1.dat")
# Cargamos el video
input_video = cv2.VideoCapture('resources/session3/test.mp4')
n_frames_used = 400 # 400 / 25 = 16 segundos

list_descriptores = []
list_images = []

print('Processing: ', end='')

for num_frame in range(n_frames_used):
    ret, frame = input_video.read()
    # Convertimos la imágen de BGR(OpenCV) a RGB(Dlib)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detectamos las caras
    bboxes = facedetector(frame, 1)
    # Iteramos sobre las caras encontradas y calculamos el vector
    for k, bbox in enumerate(bboxes):
        landmarks = facelandmark(frame, bbox)
        face_descriptor = facerembedder.compute_face_descriptor(frame, landmarks)
        list_descriptores.append(face_descriptor)
        crop = frame[bbox.top():bbox.bottom(), bbox.left():bbox.right(), :]
        list_images.append(transform.resize(crop, (45, 45)))
        
    if num_frame % 25 == 0:
        print('.', end='')

In [ ]:
from sklearn.decomposition import PCA # Algoritmo que nos permite describir un dataset con nuevas variables no correlacionadas
from sklearn.cluster import DBSCAN # Algoritmo que permíte clusterizar los datos

# Convertimos la lista de vectores en una matriz 
list_descriptores = np.array(list_descriptores)
print("Dimensiones de la matrix: ", (list_descriptores.shape))

# Reducimos las dimensiones
pca = PCA(n_components=2).fit(list_descriptores)
pca_2d = pca.transform(list_descriptores)

# Buscamos clusters
dbscan = DBSCAN(eps=0.1, min_samples=5)
dbscan.fit(pca_2d)



In [ ]:
# Mostramos el gráfico con los resultados de DBSCAN
plt.scatter(pca_2d[:, 0], pca_2d[:, 1], c=dbscan.labels_)
plt.show()

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

fig, ax = plt.subplots(figsize=(20, 10))
artists = []
for x, y, image in zip(pca_2d[:, 0], pca_2d[:, 1], list_images):
    img = OffsetImage(image, zoom=0.6)
    ab = AnnotationBbox(img, (x, y), xycoords='data', frameon=False)
    ax.add_artist(ab)
ax.set_xlim(-0.3, 0.5)
ax.set_ylim(-0.3, 0.6)
plt.show()

# Entrenamos el clasificador

Hasta este punto hemos visto cómo detectar los landmarks y como calcular un vector que describe un rostro, ¿cómo hacemos hora para reconocer cada rostro a cual corresponde?

Básicamente entrenamos un clasificador parecido al que hicimos en la segunda sesión pero en este caso capaz de etiquetar múltiples categorías, una por persona que queramos reconocer.

## Recolectando muestras

El primer paso es generar muestras para nuestro clasificador, para ello usaremos los primeros n frames para detectar caras y guardar los crops de estos. Luego manualmente los divideremos en varios directorios una para cada uno de los actores que queremos reconocer.

In [ ]:
face_detector = dlib.get_frontal_face_detector()
face_landmark = dlib.shape_predictor('resources/session3/shape_predictor_68_face_landmarks.dat')
face_embedder = dlib.face_recognition_model_v1("resources/session3/dlib_face_recognition_resnet_model_v1.dat")
input_video = cv2.VideoCapture('resources/session3/test.mp4')
n_frames_utilitzados = 900

list_descriptores = []

num_box = 0
for num_frame in range(n_frames_utilitzados):
    # Leer 
    ret, frame = input_video.read()
    # Convertimos la imágen de BGR(OpenCV) a RGB(Dlib)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.uint8)
    # Detectamos las caras
    bboxes = facedetector(frame, 1)
    for bbox in bboxes:
        io.imsave("resources/session3/our_dataset/unannotated/" + str(num_box) + ".jpg",
                  frame[bbox.top():bbox.bottom(), bbox.left():bbox.right(), :])
        num_box += 1

## Embbeded vector

Ahora que ya tenemos los diferentes personajes a reconocer divididos en carpetas, procedemos a calcular los descriptores

In [ ]:
import glob

facelandmark = dlib.shape_predictor('resources/session3/shape_predictor_68_face_landmarks.dat')
facerembedder = dlib.face_recognition_model_v1("resources/session3/dlib_face_recognition_resnet_model_v1.dat")

X = []
Y = []

nombre_carpeta = "resources/session3/our_dataset/david/"
for filename in glob.glob(nombre_carpeta + '*.jpg'):
    # Cargar muestra
    image = io.imread(filename)
    landmarks = facelandmark(image, dlib.rectangle(0, 0, image.shape[0], image.shape[1]))
    face_descriptor = facerembedder.compute_face_descriptor(image, landmarks)
    X.append(face_descriptor)
    Y.append(1)
    
nombre_carpeta = "resources/session3/our_dataset/emma/"
for filename in glob.glob(nombre_carpeta + '*.jpg'):
    # Cargar muestra
    image = io.imread(filename)
    landmarks = facelandmark(image, dlib.rectangle(0, 0, image.shape[0], image.shape[1]))
    face_descriptor = facerembedder.compute_face_descriptor(image, landmarks)
    X.append(face_descriptor)
    Y.append(2)
    
nombre_carpeta = "resources/session3/our_dataset/lopez/"
for filename in glob.glob(nombre_carpeta + '*.jpg'):
    # Cargar muestra
    image = io.imread(filename)
    landmarks = facelandmark(image, dlib.rectangle(0, 0, image.shape[0], image.shape[1]))
    face_descriptor = facerembedder.compute_face_descriptor(image, landmarks)
    X.append(face_descriptor)
    Y.append(3)

## Entrenar un clasificador

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
descriptors_scaled = scaler.transform(X)

# Dividimos el dataeset en dos partes entrenamiento/test.
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=0)

In [ ]:
# Esto es un CTRL-C + CTRL-V de los tutorial de Sklearn: 
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}]#,
                    #{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score, n_jobs=4)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
labels_names = {1: "David", 2: "Emma", 3: "Lopez"}
facedetector = dlib.get_frontal_face_detector()
# facedetector = dlib.cnn_face_detection_model_v1("resources/session2/mmod_human_face_detector.dat")
facelandmark = dlib.shape_predictor('resources/session3/shape_predictor_68_face_landmarks.dat')
facerembedder = dlib.face_recognition_model_v1("resources/session3/dlib_face_recognition_resnet_model_v1.dat")
input_video = cv2.VideoCapture('resources/session3/test.mp4')

# Escojemos un frame al azar
n_frames_used = np.random.randint(3200)
for i in range(n_frames_used):
    ret, frame = input_video.read()
    
# Lo convertimos a RGB
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# Detectamos las caras
bboxes = facedetector(frame, 1)
# Iteramos sobre las caras encontradas
for k, bbox in enumerate(bboxes):
    # Calculamos el decriptor
    landmarks = facelandmark(frame, bbox)
    face_descriptor = np.array(facerembedder.compute_face_descriptor(frame, landmarks)).reshape(1, -1)
    # Lo normalizamos
    descriptors_scaled = scaler.transform(face_descriptor)
    # Inferimos el personaje con nuestro clasificador
    label = clf.predict(descriptors_scaled)
    # Dibujamos un bbox y su nombre
    cv2.rectangle(frame,(bbox.left(), bbox.top()), (bbox.right(), bbox.bottom()), (255, 0, 0) )
    cv2.putText(frame, "#" + labels_names[label[0]], (bbox.left(), bbox.top() - 5), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)
    
# Mostramos la imágen original
plt.figure(figsize=(50, 50));
plt.imshow(frame);